<h1 style="color:#5A8E00; font-weight: bold;">
Algorithme A\*
</h1>
<br>Nous allons de ce pas nous immerger dans une application de l'algorithme A\*. Le but du script qui suit va être de déterminer les affectations qui minimisent le coût total d'un opération à partir d'un tableau à 2 dimensions dans lequel sont représentés les-dits coûts.

On importe numpy pour réaliser des opérations plus vite que l'éclair, pandas pour afficher les résultats de manière élégante et heapq qui n'est autre qu'une implémentation des Tas (qui vont nous permettre de gagner un peu / pas mal de temps de calcul). Un petit coup d'oeil sur la documentation de heapq et notamment des fonctions <b style="color:#DB5B30">heappush</b> et <b style="color:#DB5B30">heappop</b> ne fait pas de mal : https://docs.python.org/3/library/heapq.html

In [1]:
import numpy as np
import pandas as pd
from heapq import heappush, heappop

La classe Noeud ci-dessous représente une bonne partie de l'algorithme et mérite quelques éclaircissements. Nous vons invitons tout d'abord à jeter un oeil à la classe afin d'en notifier les principales attributs et fonctions.

<u style="color:#5A8E00; font-size:16px">Attributs :</u>
* <b style="color:#DB5B30">n :</b> La profondeur maximale du graphe. Si le tableau de coûts est de dimensions m\*m alors n=m
* <b style="color:#DB5B30">g :</b> Le coût d'un chemin optimal du premier noeud à celui-ci (du noeud de profondeur 0 au noeud de profondeur p)
* <b style="color:#DB5B30">p :</b> La profondeur du noeud, c'est à dire le nombre d'ancêtres que possède ce noeud. Par exemple le premier noeud est de profondeur 0, et un noeud but est de profondeur n.
* <b style="color:#DB5B30">affectations :</b> Une liste contenant toutes les affectations "qui sont faîtes" pour ce noeud. Pour un noeud but on a une liste d'affectations complète qui décrira une situation d'allocations finie. Notre algorithme devant se terminer toujours par des affectations optimales, le dernier noeud choisi possèdera donc la liste des allocations optimales.
* <b style="color:#DB5B30">h :</b> L'heuristique du noeud. C'est une estimation du coût chemin optimal de ce noeud (de profondeur p) à un but (de profondeur n). Comme nous voulons trouver le chemin optimal de 0 à n, nous devons respecter la propriété $h(p) \leq h*(p)$ où h*(p) est le coût du chemin optimal de ce noeud à un but.
* <b style="color:#DB5B30">f :</b> Estimation du coût d'un chemin optimal de 0 à un but en passant par ce noeud. C'est donc ce paramètre là que l'on cherche à minimiser. Quand l'algorithme se termine, on veut que le noeud but sur lequel il se termine soit celui avec le plus petit f possible. Ce paramètre va donc nous servir à ordonner les noeuds.

<u style="color:#5A8E00; font-size:16px">Fonctions :</u>
* <b style="color:#DB5B30">n :</b> La profondeur maximale du graphe. Si le tableau de coûts est de dimensions m\*m alors n=m
* <b style="color:#DB5B30">g :</b> Le coût d'un chemin optimal du premier noeud à celui-ci (du noeud de profondeur 0 au noeud de profondeur p)

In [2]:
class Noeud:
    def __init__(self, n, couts, g=0, p=0, affectations=list()):
        self.n = n
        self.g = g
        self.p = p
        self.affectations = affectations
        self.h = self.calculer_h(couts)
        self.f = self.g + self.h
    
    def estBut(self):
        return self.p == self.n
    
    def calculer_h(self, couts):
        if self.estBut():
            return 0
        temp = np.delete(couts[self.p:], self.affectations, 1)
        return max(np.amin(temp, 1).sum(), np.amin(temp, 0).sum())
    
    def developpe(self, couts):
        heritiers = list()
        for i, cout in enumerate(couts[self.p]):
            if i not in self.affectations:
                heritiers.append(Noeud(self.n, couts, g=self.g+cout, p=self.p+1, affectations=self.affectations+[i]))
        return heritiers
    
    def __lt__(self, other):
        return self.f < other.f

In [6]:
couts = np.random.randint(1000, size=(15, 15))

n = len(couts)
OUVERT = [Noeud(n, couts)]

while len(OUVERT) > 0:
    noeud = heappop(OUVERT)
    if noeud.estBut():
        break
    for heritier in noeud.developpe(couts):
        heappush(OUVERT, heritier)

234 ms ± 44.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
def print_affectations(couts, affectations):
    df = pd.DataFrame(couts)
    return df.style.apply(lambda tab: color_couts(affectations), axis=None)

def color_couts(affectations):
    n = len(affectations)
    ar = [['']*n for i in range(n)]
    for i, j in enumerate(affectations):
        ar[i][j] = 'background-color: #FFC900'
    return pd.DataFrame(ar)

In [5]:
print_affectations(couts, noeud.affectations)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,856,61,750,290,546,961,58,725,107,190,82,731,923,127,381
1,785,540,782,691,419,164,722,806,964,373,610,131,603,873,552
2,935,623,878,248,480,738,655,481,672,538,18,508,741,562,328
3,779,754,638,194,929,744,894,255,809,730,764,341,482,469,540
4,74,738,537,590,587,219,490,928,646,152,111,512,897,507,247
5,266,271,932,525,535,769,514,66,284,377,501,661,483,816,154
6,841,995,291,731,790,528,38,837,95,539,236,945,983,384,180
7,162,899,68,710,159,598,62,442,353,540,568,89,427,69,836
8,154,877,221,935,343,807,745,936,315,976,794,912,673,392,835
9,202,379,545,27,95,379,849,69,929,881,725,798,276,154,290


<table style="float:left; font-family:'Arial'">
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
  </tr>
</table>